In [2]:
def Preprocess(file):
    array = []
    for line in f:
        array.append(line)
    xc = [0]*300
    yc = [0]*300
    maxrows = 300
    maxcols = 300
    Cost = [([0] * maxcols) for row in range(maxrows) ]

    dim = int(array[0])
    for it in range(1, dim+1):
        p = array[it]
        tokens = p.split()
        xc[it-1] = float(tokens[0])
        yc[it-1] = float(tokens[1])
    
    for i in range(0, dim):
        for j in range(0, dim):
            Cost[i][j] = pow((pow(xc[i]-xc[j], 2) + pow(yc[i] - yc[j], 2)), 0.5)
        
    for i in range(0, dim):
        for j in range(0, dim):
            if(i==j):
                Cost[i][j] = 99999
    return Cost, xc, yc, dim

In [3]:
from gurobipy import *
import string
import time

f = open("C:\\Users\\Prasangsha\\Downloads\\TSP_instance_n_30_s_1.dat")
Cost, xc, yc, dim = Preprocess(f)

startTime = time.time()

TSP_MCF = Model('TSP_MultiCommodity_Flow')

d = [[0 for i in range(0, dim)] for k in range(0, dim)] 

for i in range(0, dim):
    for k in range(0, dim):
        if(i == 0):
            d[i][k] = 1
        if(k == i):
            d[i][k] = -1
            

x = [[0 for i in range(dim)] for j in range(dim)] 
for i in range(0, dim):
    for j in range(0, dim):
        x[i][j] = TSP_MCF.addVar(vtype=GRB.BINARY, name = "x[][]".format(i,j))

y = [[[0 for i in range(dim)] for j in range(dim)] for k in range(dim)] 
for i in range(0, dim):
    for j in range(0, dim):
        for k in range(0, dim):
            y[i][j][k] = TSP_MCF.addVar(vtype=GRB.BINARY, name = 'u[]'.format(i))
    
TSP_MCF.update()

TSP_MCF.setObjective(quicksum(quicksum(Cost[i][j]*x[i][j] for i in range(dim)) for j in range(dim)), GRB.MINIMIZE)



for i in range(0, dim):
    TSP_MCF.addConstr(quicksum(x[i][j] for j in range(0, dim) ) == 1)
    
for j in range(0, dim):
    TSP_MCF.addConstr(quicksum(x[i][j] for i in range(0, dim) ) == 1)
    

for i in range(0, dim):
    for k in range(1, dim):
        TSP_MCF.addConstr(quicksum(y[i][j][k] for j in range(0, dim)) - quicksum(y[j][i][k] for j in range(0, dim) ) == d[i][k])
        
for i in range(0, dim):
    for k in range(0, dim):
        for j in range(0, dim):
            TSP_MCF.addConstr(y[i][j][k] <= x[i][j])
            
for i in range(0, dim):
    for k in range(0, dim):
        for j in range(0, dim):
            TSP_MCF.addConstr(y[i][j][k] >= 0)
            

TSP_MCF.optimize()

executionTime = (time.time() - startTime)
print("The execution time is")
print(executionTime)


Using license file C:\Users\Prasangsha\gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 54930 rows, 27900 columns and 133260 nonzeros
Model fingerprint: 0xfdcc72ca
Variable types: 0 continuous, 27900 integer (27900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 32134 rows and 5106 columns
Presolve time: 2.38s
Presolved: 22796 rows, 22794 columns, 91060 nonzeros
Variable types: 0 continuous, 22794 integer (22794 binary)
Found heuristic solution: objective 355.2111739
Found heuristic solution: objective 286.5090164

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   19764    8.9008439e+01   0.000000e+00   0.000000e+00      5s

Root relaxation: objective 8.900825e+01, 19764 iterations, 2.45 seconds

    Nodes    |    Current Node   